# AutoGen 基本サンプル

このコードサンプルでは、[AutoGen](https://aka.ms/ai-agents/autogen) AIフレームワークを使用して基本的なエージェントを作成します。

このサンプルの目的は、後のコードサンプルで異なるエージェントパターンを実装する際に使用する手順を示すことです。


## 必要なPythonパッケージをインポート


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## クライアントの作成

このサンプルでは、LLMにアクセスするために[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`model`は`gpt-4o-mini`として定義されています。GitHub Modelsマーケットプレイスで利用可能な別のモデルに変更して、異なる結果を確認してみてください。

簡単なテストとして、シンプルなプロンプトを実行します - `フランスの首都は何ですか`


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## エージェントの定義

`client`を設定し、それが正常に動作していることを確認したので、次に`AssistantAgent`を作成します。各エージェントには以下を割り当てることができます：
**name** - 複数エージェントのフローで参照する際に便利な短い名前。  
**model_client** - 前のステップで作成したクライアント。  
**tools** - エージェントがタスクを完了するために使用できるツール。  
**system_message** - LLMのタスク、振る舞い、トーンを定義するメタプロンプト。

システムメッセージを変更して、LLMの応答がどのように変わるかを確認できます。`tools`についてはレッスン#4で取り上げます。


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## エージェントを実行する

以下の関数はエージェントを実行します。`on_message` メソッドを使用して、新しいメッセージでエージェントの状態を更新します。

この場合、ユーザーからの新しいメッセージである「素晴らしい晴れた休暇を計画して」という内容で状態を更新します。

メッセージの内容を変更することで、LLMがどのように異なる応答をするかを確認できます。


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤認について、当方は責任を負いません。
